In [ ]:
#uses Registered2to1Adder_NIR - v1.1
#OR and registerings implemented based on the assumption that delay of Registered2to1Adder_NIR - v1.1 is 1 clk

In [ ]:
import sys
import math
import io

In [ ]:
#N>=2
N = input('Enter N: ')

In [ ]:
#IN_WIDTH>=1
IN_WIDTH = input('Enter IN_WIDTH default value (empty for 10): ')

In [ ]:
N = int(N)
if N<2:
    N=2
if not IN_WIDTH:
    IN_WIDTH=10
else:
    IN_WIDTH = int(IN_WIDTH)
    if IN_WIDTH<1:
        IN_WIDTH=1

In [ ]:
__Print_To_File = 0

if __Print_To_File<=0:
    of=sys.stdout
else:
    of=open("./AdderTree_{}.v".format(N), 'w+')

In [ ]:
print("`timescale 1ns / 1ps\n", file=of)
print("module AdderTree_{}".format(N), file=of)
print("#(", file=of)
print("parameter IN_WIDTH = {}".format(IN_WIDTH), file=of)
print(")(", file=of)
print("input clk, reset, enable,", file=of)
print("input inReady,", file=of)
print("input signed [IN_WIDTH-1:0] ", end='', file=of)
for i in range(N):
    print("A{}, ".format(i), end='', file=of)
print(file=of)
print("output outReady,", file=of)
lgn=math.ceil(math.log(N,2))
al=-1+lgn
if al==0:
    print("output signed [IN_WIDTH:0] Out,", file=of)
else:
    print("output signed [IN_WIDTH+{}:0] Out,".format(al), file=of)
print("output earlyOutReady", file=of)
print(");\n", file=of)

In [ ]:
regsBuf=io.StringIO()
AIBuf=io.StringIO()

lgN=math.ceil(math.log2(N))
maxLevel=lgN+1
sizeA=[]
for j in range(N):
    sizeA.append(0)
i=0
level=1
maxn=N-1
new=maxn
while 1 :
    if level==maxLevel :
        break
    if i<maxn :
        new=new+1
        sizeA.append(sizeA[i]+1) #will be 1 at least
        #if sizeA[new]==0:
        #    AIBuf.write("wire signed [IN_WIDTH-1:0] M{};\n".format(new))
        #el
        if sizeA[i]==0:
            AIBuf.write("wire signed [IN_WIDTH:0] M{};\n".format(new))
            AIBuf.write("Registered2to1Adder_NIR #(.IN_WIDTH(IN_WIDTH))\nAdd{}(\nclk, reset, enable, ".format(new))
            AIBuf.write("inReady,\nA{}, A{}".format(i, i+1))
        else:
            AIBuf.write("wire signed [IN_WIDTH+{}:0] M{};\n".format(sizeA[i], new))
            esize=sizeA[i]-sizeA[i+1]
            if(esize!=0):
                if sizeA[i]==1:
                     AIBuf.write("wire signed [IN_WIDTH:0] M{}e=".format(i+1))
                else:
                     AIBuf.write("wire signed [IN_WIDTH+{}:0] M{}e=".format(sizeA[i]-1, i+1))
                AIBuf.write("{{")
                AIBuf.write("{}".format(esize))
                AIBuf.write("{")
                AIBuf.write("M{}[IN_WIDTH".format(i+1))
                if sizeA[i+1]==0:
                    AIBuf.write("-1")
                elif sizeA[i+1]>1:
                    AIBuf.write("+{}".format(sizeA[i+1]-1))
                AIBuf.write("]}},")
                AIBuf.write("M{}".format(i+1))
                AIBuf.write("};\n")
            AIBuf.write("Registered2to1Adder_NIR #(.IN_WIDTH(IN_WIDTH+{}))\nAdd{}(\nclk, reset, enable, ".format(sizeA[i], new))
            AIBuf.write("OR[{}],\nM{}, M{}".format(level-2, i, i+1))
            if(sizeA[i+1]!=sizeA[i]):
                AIBuf.write("e");
        AIBuf.write(",\naor{}, //not used\nM{},\naeor{}); //not used\n\n".format(new, new, new))
        #if level==1:
        #else :
        if (i+1)==maxn :
            maxn=new
            level=level+1
        i=i+2
    else :
        new=new+1
        sizeA.append(sizeA[i])
        if sizeA[new]==0:
            print("reg signed [IN_WIDTH-1:0] M{};".format(new), file=of)
        elif sizeA[new]==1:
            print("reg signed [IN_WIDTH:0] M{};".format(new), file=of)
        else:
            print("reg signed [IN_WIDTH+{}:0] M{};".format(sizeA[new]-1, new), file=of)
        if level==1:
            regsBuf.write("\t\tif(inReady) begin\n")
            regsBuf.write("\t\t\tM{} <= A{};\n".format(new, i))
            regsBuf.write("\t\tend\n")
        else:
            regsBuf.write("\t\tif(OR[{}]) begin\n".format(level-2))
            regsBuf.write("\t\t\tM{} <= M{};\n".format(new, i))      
            regsBuf.write("\t\tend\n")
        i=i+1
        maxn=new
        level=level+1

In [ ]:
print("reg [0:{}] OR = 0;".format(lgN-1), file=of)
print("integer d;", file=of)
print("always @(posedge clk) begin", file=of)
print("\tif(reset) begin", file=of)
print("\t\tOR <= {}'b0;".format(lgN), file=of)
print("\tend", file=of)
print("\telse if(enable) begin", file=of)
print("\t\tOR[0] <= inReady;", file=of)
if lgN>1:
    print("\t\tfor(d=0;d<{};d=d+1) begin".format(lgN-1), file=of)
    print("\t\t\tOR[d+1] <= OR[d];", file=of)
    print("\t\tend", file=of)
print(regsBuf.getvalue(), end='',file=of)
print("\tend", file=of)
print("end\n", file=of)

In [ ]:
print(AIBuf.getvalue(), end='',file=of)
print("assign Out = M{};".format(new), file=of)
print("assign outReady = OR[{}];".format(lgN-1), file=of)
if N==2:
    print("assign earlyOutReady = inReady;", file=of)
else:
    print("assign earlyOutReady = OR[{}];".format(lgN-2), file=of)
print("\nendmodule", file=of)

In [ ]:
if __Print_To_File>0:
    of.close()